### mysql와 python의 연동
 1. 라이브러리 설치
 2. sql 서버의 정보 입력
 3. sql서버에 접속
 4. sql 쿼리문을 이용하여 데이터를 CRUD 작업

In [1]:
### mysql라이브러리 설치
!pip install pymysql

     ---------------------------------------- 43.8/43.8 kB 1.0 MB/s eta 0:00:00


In [3]:
import pymysql
import pandas as pd


In [6]:
## sql 서버 정보 입력   -> DB접속
_db = pymysql.connect(
    user = 'root',
    password = '1234',
    host =  'localhost',
    db = 'ubion',
    port = 3306
)

In [8]:
## cursor 변수 
## sql 쿼리문을 실행 시키기 위해서 만들어야 되는 필수 변수
## DicCursor는 sql쿼리문을 실행되고 나온 결과물을 딕셔너리 형태로 받겠다.
cursor = _db.cursor(pymysql.cursors.DictCursor)       

In [92]:
sql = """
        select  
        *
        from 
        user_info
      """

cursor.execute(sql)           

result = cursor.fetchall()    ## 결과값 가져오기
print(result)

[{'id': '1', 'name': '2', 'age': 3, 'address': '4'}, {'id': '3145', 'name': '1243', 'age': 123, 'address': '124'}, {'id': 'test', 'name': 'moon', 'age': None, 'address': 'ubion'}, {'id': 'test11', 'name': 'park', 'age': 40, 'address': 'ubion10'}, {'id': 'test2', 'name': 'kim', 'age': 30, 'address': 'ubion'}, {'id': 'test4', 'name': 'park', 'age': 30, 'address': 'ubion2'}, {'id': 'test9', 'name': 'lee', 'age': 25, 'address': 'ubion5'}]


In [13]:
df = pd.DataFrame(result)
df

,ID,NAME,AGE,ADDRESS
0,test,jihun,None,UBION


In [18]:
sql = """
        insert into
        user_info(id,name,age)
        values
        ('test2','kim',30)
    """
cursor.execute(sql)


1

In [20]:
_db.commit()  ## 변경한 데이터 저장, 적용


In [26]:
sql = """
        select 
        createDt as 날짜,
        deathCnt as 시망자,
        decideCnt as 확진자
        from
        corona
        order by
        createDt
"""
cursor.execute(sql)
result = cursor.fetchall()
df =pd.DataFrame(result)
df

,날짜,시망자,확진자
0,2020-01-20 17:00:00.000,0,1
1,2020-01-21 09:00:00.000,0,1
2,2020-01-22 12:00:00.000,0,1
3,2020-01-23 09:00:00.000,0,1
4,2020-01-24 00:00:00.000,0,2
...,...,...,...
1054,2022-12-10 00:41:17.0,31029,27674163
1055,2022-12-11 01:10:06.0,31069,27728482
1056,2022-12-12 00:34:36.0,31099,27754149
1057,2022-12-13 04:34:20.0,31128,27841001


### sql 쿼리문 실행 함수 생성
1. select() 함수 생성 
2. 매개변수는 2개
3. 매개변수 sql은 sql 쿼리문을 입력받는 변수
4. 매개변수 limit는 기본값은 0 
5. limit가 0인 경우는 쿼리문의 결과를 전부 보여주고 0이 아닌 경우에는 limit의 개수만큼 행을 보여준다.
6. sql 쿼리문을 받아서 실행하고 결과값을 받아서 return

In [45]:
##case1
def select(sql,limit=0):
    cursor.execute(sql)
    result = cursor.fetchall()
    if limit !=0:
        result =result[:limit]
    df = pd.DataFrame(result)
    return df


In [49]:
##case2
def select(sql,limit=0):
    cursor.execute(sql)
    result = cursor.fetchall()
    df = pd.DataFrame(result)
    if limit==0:
        return df
    else:
        return df.head(limit)


In [62]:
#case3
def select(sql,limit=0):
    if limit != 0:
        sql = sql + "limit"+ str(limit)
    cursor.execute(sql)
    result = cursor.fetchall()
    df = pd.DataFrame(result)
    return df


In [65]:

select(sql,2)

,createDt,deathCnt,decideCnt,seq,stateDt,stateTime,updateDt,accExamCnt,accDefRate
0,2022-12-14 00:42:41.0,31174,27925572,1099,20221214,00:00,None,,
1,2022-12-13 04:34:20.0,31128,27841001,1098,20221213,00:00,None,,
2,2022-12-12 00:34:36.0,31099,27754149,1097,20221212,00:00,None,,
3,2022-12-11 01:10:06.0,31069,27728482,1096,20221211,00:00,None,,
4,2022-12-10 00:41:17.0,31029,27674163,1095,20221210,00:00,None,,
...,...,...,...,...,...,...,...,...,...
1054,2020-01-24 00:00:00.000,0,2,5,20200124,00:00,2021-10-07 10:30:51.51,29,6.896551724
1055,2020-01-23 09:00:00.000,0,1,4,20200123,09:00,2021-10-07 10:30:51.51,25,4
1056,2020-01-22 12:00:00.000,0,1,3,20200122,12:00,2021-10-07 10:30:51.51,21,4.761904762
1057,2020-01-21 09:00:00.000,0,1,2,20200121,09:00,2021-10-07 10:30:51.51,15,6.666666667


### input 을 이용하여 입력받은 데이터 값을 insert
1. input 함수를 이용하여 데이터를 4개받는다
2. insert문을 이용하여 user_info테이블에 ID,NAME,AGE,ADRESS 항목에 삽입

In [83]:
#case1
_id = input("id: ")
_name = input("NAME: ")
_age = input("AGE :")
_adress = input("ADRESS :" )
sql = """
        insert into
        user_info
        values
        (%s,%s,%s,%s)

    """
cursor.execute(sql,[_id,_name,_age,_adress])
_db.commit()

In [85]:
#case2
_id = input("id: ")
_name = input("NAME: ")
_age = input("AGE :")
_adress = input("ADRESS :" )
sql = f"""                         
        insert into
        user_info
        values
        ('{_id}','{_name}','{_age}','{_adress}')

    """
cursor.execute(sql)
_db.commit()

In [87]:
def insert(sql, *data):          ### *변수의 개수를 정하지 않음
    cursor.execute(sql,data)
    _db.commit()
    return cursor.fetchall()

sql = """
        insert into
        user_info
        values
        ()
    """    

a ['v', 'd', 'E']


In [102]:
## 데이터베이스와의 접속을 종료

_db.close()

In [91]:
## 외부의 있는 서버와의 접속
## root아이디는 내부에서만 접속이 가능한 아이디
_db = pymysql.connect(
    user = "ubion",               ## ubion이라는 다른 계정으로 로그인
    password = "1234",
    host = "172.16.12.149",       ## 와이파이 아이피
    db = "ubion",
    port =3306
)

cursor = _db.cursor(pymysql.cursors.DictCursor)

In [101]:
sql = """
        select  
        *
        from 
        user_info
      """

cursor.execute(sql)           

result = cursor.fetchall()    ## 결과값 가져오기
print(result)

[{'id': '1', 'name': '2', 'age': 3, 'address': '4'}, {'id': '3145', 'name': '1243', 'age': 123, 'address': '124'}, {'id': 'abc', 'name': '30', 'age': 20, 'address': 'ubion11'}, {'id': 'test', 'name': 'moon', 'age': None, 'address': 'ubion'}, {'id': 'test10', 'name': 'new', 'age': 99, 'address': 'house'}, {'id': 'test11', 'name': 'park', 'age': 40, 'address': 'ubion10'}, {'id': 'test2', 'name': 'kim', 'age': 30, 'address': 'ubion'}, {'id': 'test4', 'name': 'park', 'age': 30, 'address': 'ubion2'}, {'id': 'test9', 'name': 'lee', 'age': 25, 'address': 'ubion5'}]


In [98]:
sql = """                         
        insert into
        user_info
        values
        ('test10','new','99','house')

    """
cursor.execute(sql)
_db.commit()

IntegrityError: (1062, "Duplicate entry 'test10' for key 'user_info.PRIMARY'")

In [ ]:
_db.close()